In [1]:
import os, pickle, yaml, sys, torch, random, math, librosa, pdb
from torch.backends import cudnn
import soundfile as sf
from synthesis.synthesis import build_model
from synthesis.synthesis import wavegen
import numpy as np
import utils
import pyworld as pw
from model_vc import Generator
from utils import setup_sie, setup_gen
sys.path.insert(1, '/homes/bdoc3/my_utils')
from audio.worldvocoder import chandna_feats, mfsc_to_sp, midi_to_worldf0, mfsc_to_world_to_audio
from my_os import recursive_file_retrieval
from my_arrays import fix_feat_length

In [3]:
# parser data
svc_model_name = 'withF0chandna_to_500_unnormed'
svc_model_dir = '/homes/bdoc3/my_data/autovc_models/autoSvc'
svc_ckpt_iters = 252500
which_cuda = 1
feature_dir = '/homes/bdoc3/my_data/world_vocoder_data/damp_inton/withF0chandna_to_500_unnormed'

In [4]:
# determine relevant paths and yaml files based on user input
autovc_model_dir = os.path.join(svc_model_dir, svc_model_name)
autosvc_config = pickle.load(open(autovc_model_dir +'/config.pkl','rb'))
autosvc_config.autovc_ckpt = os.path.join(autovc_model_dir, 'ckpts', f'ckpt_{svc_ckpt_iters}.pth.tar')
with open(feature_dir +'/feat_params.yaml') as File:
    feat_params = yaml.load(File, Loader=yaml.FullLoader)
subdir_for_wavs = os.path.join(autovc_model_dir, 'generated_wavs')
if os.path.exists(subdir_for_wavs)==False:
            os.makedirs(subdir_for_wavs)

# determine other derivative variables
autosvc_which_cuda = which_cuda
autosvc_config.device = torch.device(f'cuda:{autosvc_which_cuda}')

In [23]:
file_path = '/import/c4dm-datasets/NUS_corpus/MPOL/sing/19.wav'

feats_ids = ['1014969323_1909869121', '1026003246_1660051761'] # female alto (low) and falsetto tenor (high)
dataset_root = '/import/c4dm-datasets/DAMP_Intonation_Dataset/vocal_tracks'
file_ext = '.m4a'

feats_paths = [os.path.join(feature_dir, 'val', a_id.split('_')[0], a_id)+'.npy' for a_id in feats_ids]

user_input = '/homes/bdoc3/my_data/world_vocoder_data/damp_inton/withF0chandna_to_500_unnormed/val'






# if input is not file type
if os.path.isdir(user_input):
    subdir_list, file_path_list = recursive_file_retrieval(user_input)
    selected_file_paths = random.choices(file_path_list, k=num_files)
    user_input = selected_file_paths
else:
    file_paths = [user_input]

feats_list = []
for file in file_paths
    if not file.endswith('.npy'):
        # assume its audio file
        audio, _ = librosa.load(file_path, sr=feat_params['sr'])
        feats = chandna_feats(formatted_audio.astype('double'), feat_params)
        feats_list.append(feats)
    else:
        feats_list.append(np.load(file))


['/homes/bdoc3/my_data/world_vocoder_data/damp_inton/withF0chandna_to_500_unnormed/val/1198871647/1198871647_1859856826.npy', '/homes/bdoc3/my_data/world_vocoder_data/damp_inton/withF0chandna_to_500_unnormed/val/575984180/575984180_1660276054.npy', '/homes/bdoc3/my_data/world_vocoder_data/damp_inton/withF0chandna_to_500_unnormed/val/1224937687/1224937687_1849548794.npy', '/homes/bdoc3/my_data/world_vocoder_data/damp_inton/withF0chandna_to_500_unnormed/val/1427897033/1427897033_1721850720.npy']


In [24]:
# sys.path.insert(1, autovc_model_dir)
# from this_model_vc import Generator

G = setup_gen(autosvc_config, Generator, feat_params['num_feats'])
sie = setup_sie(autosvc_config)

In [33]:
random_offsets = []
autosvc_svc_timesteps = 160

def infer(feats_paths, si_encoder, autosvc_G):
    "feeds features into framework and outputs list containing outputs and paths"
    feats_list = [torch.from_numpy(np.load(feat_path)).to(autosvc_config.device).float() for feat_path in feats_paths] # convert to torch objects
    outputs_sources_list = []
    for trg_path, trg_feats in zip(feats_paths, feats_list):
        feats_numpy = trg_feats.cpu().detach().numpy()
        synthed_audio = mfsc_to_world_to_audio(feats_numpy[:,:feat_params['num_feats']-4], feats_numpy[:,feat_params['num_feats']-4:-2], feats_numpy[:,-2:], feat_params)
        sf.write(subdir_for_wavs +f'/{os.path.basename(trg_path)[:-4]}.wav', synthed_audio, samplerate=feat_params['sr'])
        
        for src_path, src_feats in zip(feats_paths, feats_list):
            
#             trimmed_feats_list = [trim_feat_length(feats, autosvc_svc_timesteps) for feats in [trg_feats, src_feats]]
#             spec_env_list = [trimmed_feats[:,:feat_params['num_feats']] for trimmed_feats in trimmed_feats_list]
            trimmed_trg_feats = trim_feat_length(trg_feats, autosvc_svc_timesteps)
            trimmed_src_feats = trim_feat_length(src_feats, autosvc_svc_timesteps)
            spec_env_trg = trimmed_trg_feats[:,:feat_params['num_feats']]
            spec_env_src = trimmed_src_feats[:,:feat_params['num_feats']]
            src_pitch_feats = trimmed_src_feats[:,feat_params['num_feats']:]
            src_pitch_feats_np = src_pitch_feats.cpu().detach().numpy()
            si_emb, _ = si_encoder(spec_env_trg.unsqueeze(0))
            _, psnt_output, _, _, _ = autosvc_G(spec_env_src.unsqueeze(0), si_emb, si_emb)
            psnt_output_np = psnt_output.cpu().detach().numpy().squeeze(0).squeeze(0)
#             print('inner', trg_path, src_path)
            outputs_sources_list.append((psnt_output_np, src_pitch_feats_np, trg_path, src_path))
            
        trimmed_trg_feats = trim_feat_length(trg_feats, autosvc_svc_timesteps)
        trimmed_trg_feats_np = trimmed_trg_feats.cpu().detach().numpy()
        trg_pitch_feats_np = trimmed_trg_feats_np[:,feat_params['num_feats']:]
#         print('outer', trg_path)
        outputs_sources_list.append((trimmed_trg_feats_np, trg_pitch_feats_np, 'path/to/original.npy', trg_path))
        
    return outputs_sources_list

outputs_sources_list = infer(feats_paths, sie, G)


In [34]:
for i, (feats, src_pitch_feats, trg_path, src_path) in enumerate(outputs_sources_list):
    print(i)
    harm_mfsc, ap_mfsc = feats[:,:40], feats[:,40:44]
    midi_voicings = src_pitch_feats
#     pdb.set_trace()
    synthed_audio = mfsc_to_world_to_audio(harm_mfsc, ap_mfsc, midi_voicings, feat_params)
    trg_singer, src_singer = os.path.basename(trg_path)[:-4], os.path.basename(src_path)[:-4]
    sf.write(subdir_for_wavs +f'/-source-{src_singer}-target-{trg_singer}.wav', synthed_audio, samplerate=feat_params['sr'])
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


## WaveNet was trained on 80 mel spectrogram. not suitable for 40 mel spectral envelope

In [ ]:
voice_synth = build_model().to(autosvc_config.device)
voice_synth.load_state_dict(synth_voice_ckpt["state_dict"])
voice_synth.to(autosvc_config.device)

In [ ]:
import pickle

results = pickle.load(open('results.pkl', 'rb'))
# subdir_for_conversion = subdir_for_wavs +'/conversions'
# if os.path.exists(subdir_for_conversion)==False:
#     os.makedirs(subdir_for_conversion)
result = results[0]
name = result[0]
c = result[1]
print(name, c[:].shape, type(c))

In [ ]:

waveform = wavegen(voice_synth, 1, c=c) 

In [ ]:

  
#     librosa.output.write_wav(name+'.wav', waveform, sr=16000)
#     sf.write(subdir_for_conversion +'/' +name +'.wav', waveform, samplerate=16000)

In [ ]:
import pdb
for i, (feats, na, nb) in enumerate(outputs_sources_list):
    feats_reshaped = feats.cpu().detach().numpy().squeeze(0).squeeze(0)
    pdb.set_trace()
    waveform = wavegen(voice_synth, 1, c=feats_reshaped)
#     waveform = wavegen(voice_synth, autosvc_which_cuda, c=feats_reshaped)
    sf.write(subdir_for_wavs +f'/sourceId_{na}_to_targetId_{nb}.wav', waveform, samplerate=feat_params['sr'])
    